In [1]:
import pickle 
import os 
import torch 
from collections import Counter 
from tqdm import tqdm 

with open(os.path.expanduser("~/dataset/OAG/raw/PT-HGNN/graph_Chemistry.dict.pkl"), "rb") as fp:
    graph_info = pickle.load(fp) 
    
edge_list = graph_info['edge_list'] 
node_feature = graph_info['node_feature']
node_forward = graph_info['node_forward'] 
times = graph_info['times']

In [2]:
num_paper_nodes = len(node_feature["paper"])

num_paper_nodes

747290

In [3]:
paper_year_vec = torch.zeros(num_paper_nodes, dtype=torch.int64)

for etype in tqdm(graph_info['edge_list']['field']['paper']):
    for field_id in graph_info['edge_list']['field']['paper'][etype]: 
        for paper_id in graph_info['edge_list']['field']['paper'][etype][field_id]: 
            paper_year = graph_info['edge_list']['field']['paper'][etype][field_id][paper_id] 

            if paper_year_vec[paper_id] == 0: 
                paper_year_vec[paper_id] = paper_year 
            else: 
                assert paper_year_vec[paper_id] == paper_year 
                
(paper_year_vec > 0).sum(), (paper_year_vec > 0).float().mean() 

100%|██████████| 6/6 [01:17<00:00, 12.86s/it]


(tensor(747290), tensor(1.))

In [4]:
counter_result = sorted(Counter(paper_year_vec.tolist()).items()) 

counter_result[:10], counter_result[-10:]

([(1901, 1),
  (1909, 1),
  (1911, 2),
  (1912, 1),
  (1913, 1),
  (1916, 1),
  (1918, 1),
  (1919, 1),
  (1920, 4),
  (1923, 1)],
 [(2010, 34931),
  (2011, 41432),
  (2012, 43764),
  (2013, 48118),
  (2014, 53321),
  (2015, 62609),
  (2016, 76358),
  (2017, 59669),
  (2018, 10644),
  (2019, 7)])

In [5]:
edge_index_dict = dict() 

for src_ntype in tqdm(graph_info['edge_list']): 
    for dest_ntype in graph_info['edge_list'][src_ntype]:
        for etype in graph_info['edge_list'][src_ntype][dest_ntype]:
            edge_list = [] 
            
            for src_nid in graph_info['edge_list'][src_ntype][dest_ntype][etype]: 
                for dest_nid in graph_info['edge_list'][src_ntype][dest_ntype][etype][src_nid]: 
                    edge_list.append((src_nid, dest_nid))

            edge_index = torch.tensor(edge_list, dtype=torch.int64).T 
            
            edge_index_dict[(src_ntype, etype, dest_ntype)] = edge_index 

edge_index_dict.keys() 

100%|██████████| 5/5 [00:14<00:00,  2.92s/it]


dict_keys([('venue', 'PV_Journal', 'paper'), ('venue', 'PV_Patent', 'paper'), ('venue', 'PV_Conference', 'paper'), ('paper', 'rev_PV_Journal', 'venue'), ('paper', 'rev_PV_Patent', 'venue'), ('paper', 'rev_PV_Conference', 'venue'), ('paper', 'PP_cite', 'paper'), ('paper', 'rev_PP_cite', 'paper'), ('paper', 'rev_PF_in_L3', 'field'), ('paper', 'rev_PF_in_L2', 'field'), ('paper', 'rev_PF_in_L1', 'field'), ('paper', 'rev_PF_in_L0', 'field'), ('paper', 'rev_PF_in_L4', 'field'), ('paper', 'rev_PF_in_L5', 'field'), ('paper', 'AP_write_last', 'author'), ('paper', 'AP_write_other', 'author'), ('paper', 'AP_write_first', 'author'), ('field', 'FF_in', 'field'), ('field', 'rev_FF_in', 'field'), ('field', 'PF_in_L3', 'paper'), ('field', 'PF_in_L2', 'paper'), ('field', 'PF_in_L1', 'paper'), ('field', 'PF_in_L0', 'paper'), ('field', 'PF_in_L4', 'paper'), ('field', 'PF_in_L5', 'paper'), ('affiliation', 'in', 'author'), ('author', 'rev_in', 'affiliation'), ('author', 'rev_AP_write_last', 'paper'), ('aut

In [6]:
_edge_index_dict = dict() 

for (src_ntype, etype, dest_ntype), edge_index in edge_index_dict.items(): 
    if src_ntype == 'affiliation': 
        src_ntype = 'institution' 
    if dest_ntype == 'affiliation': 
        dest_ntype = 'institution'    
    
    if etype[:2].isupper(): 
        etype = etype[1] + etype[0] + etype[2:] 
    elif etype[:3] == 'rev': 
        etype = etype[4:]
        
    etype = etype.replace('_in_', '_')
    etype = etype.replace('_write_', '_')
        
    _edge_index_dict[(src_ntype, etype, dest_ntype)] = edge_index 
    
edge_index_dict = _edge_index_dict 

edge_index_dict.keys() 

dict_keys([('venue', 'VP_Journal', 'paper'), ('venue', 'VP_Patent', 'paper'), ('venue', 'VP_Conference', 'paper'), ('paper', 'PV_Journal', 'venue'), ('paper', 'PV_Patent', 'venue'), ('paper', 'PV_Conference', 'venue'), ('paper', 'PP_cite', 'paper'), ('paper', 'PF_L3', 'field'), ('paper', 'PF_L2', 'field'), ('paper', 'PF_L1', 'field'), ('paper', 'PF_L0', 'field'), ('paper', 'PF_L4', 'field'), ('paper', 'PF_L5', 'field'), ('paper', 'PA_last', 'author'), ('paper', 'PA_other', 'author'), ('paper', 'PA_first', 'author'), ('field', 'FF_in', 'field'), ('field', 'FP_L3', 'paper'), ('field', 'FP_L2', 'paper'), ('field', 'FP_L1', 'paper'), ('field', 'FP_L0', 'paper'), ('field', 'FP_L4', 'paper'), ('field', 'FP_L5', 'paper'), ('institution', 'in', 'author'), ('author', 'in', 'institution'), ('author', 'AP_last', 'paper'), ('author', 'AP_other', 'paper'), ('author', 'AP_first', 'paper')])

In [7]:
edge_index_dict[('institution', 'IA', 'author')] = edge_index_dict.pop(('institution', 'in', 'author'))
edge_index_dict[('author', 'AI', 'institution')] = edge_index_dict.pop(('author', 'in', 'institution'))

PP_edge_index = edge_index_dict.pop(('paper', 'PP_cite', 'paper')) 
PP_edge_index = torch.cat([PP_edge_index, torch.flip(PP_edge_index, dims=[0])], dim=-1) 
PP_edge_index = torch.unique(PP_edge_index, dim=-1) 
edge_index_dict[('paper', 'PP', 'paper')] = PP_edge_index 

{ k: v.shape for k, v in edge_index_dict.items() }

{('venue', 'VP_Journal', 'paper'): torch.Size([2, 747053]),
 ('venue', 'VP_Patent', 'paper'): torch.Size([2, 176]),
 ('venue', 'VP_Conference', 'paper'): torch.Size([2, 61]),
 ('paper', 'PV_Journal', 'venue'): torch.Size([2, 747053]),
 ('paper', 'PV_Patent', 'venue'): torch.Size([2, 176]),
 ('paper', 'PV_Conference', 'venue'): torch.Size([2, 61]),
 ('paper', 'PF_L3', 'field'): torch.Size([2, 1710391]),
 ('paper', 'PF_L2', 'field'): torch.Size([2, 3108157]),
 ('paper', 'PF_L1', 'field'): torch.Size([2, 1332023]),
 ('paper', 'PF_L0', 'field'): torch.Size([2, 747417]),
 ('paper', 'PF_L4', 'field'): torch.Size([2, 398183]),
 ('paper', 'PF_L5', 'field'): torch.Size([2, 87674]),
 ('paper', 'PA_last', 'author'): torch.Size([2, 664930]),
 ('paper', 'PA_other', 'author'): torch.Size([2, 1634514]),
 ('paper', 'PA_first', 'author'): torch.Size([2, 630999]),
 ('field', 'FF_in', 'field'): torch.Size([2, 410909]),
 ('field', 'FP_L3', 'paper'): torch.Size([2, 1710391]),
 ('field', 'FP_L2', 'paper'): 

In [9]:
PV_edge_index = torch.cat([
    edge_index_dict[('paper', 'PV_Conference', 'venue')], 
    edge_index_dict[('paper', 'PV_Journal', 'venue')], 
    edge_index_dict[('paper', 'PV_Patent', 'venue')], 
], dim=-1)

PV_edge_index = torch.unique(PV_edge_index, dim=-1) 

VP_edge_index = torch.flip(PV_edge_index, dims=[0]) 

edge_index_dict[('paper', 'PV', 'venue')] = PV_edge_index 
edge_index_dict[('venue', 'VP', 'paper')] = VP_edge_index 

In [10]:
PF_edge_index = torch.cat([
    edge_index_dict[('paper', 'PF_L5', 'field')], 
    edge_index_dict[('paper', 'PF_L4', 'field')], 
    edge_index_dict[('paper', 'PF_L3', 'field')], 
    edge_index_dict[('paper', 'PF_L2', 'field')], 
], dim=-1)

PF_edge_index = torch.unique(PF_edge_index, dim=-1) 

FP_edge_index = torch.flip(PF_edge_index, dims=[0]) 

edge_index_dict[('paper', 'PF', 'field')] = PF_edge_index 
edge_index_dict[('field', 'FP', 'paper')] = FP_edge_index 

In [11]:
PA_edge_index = torch.cat([
    edge_index_dict[('paper', 'PA_last', 'author')], 
    edge_index_dict[('paper', 'PA_other', 'author')], 
    edge_index_dict[('paper', 'PA_first', 'author')], 
], dim=-1)

PA_edge_index = torch.unique(PA_edge_index, dim=-1) 

AP_edge_index = torch.flip(PA_edge_index, dims=[0]) 

edge_index_dict[('paper', 'PA', 'author')] = PA_edge_index 
edge_index_dict[('author', 'AP', 'paper')] = AP_edge_index 

{ k: v.shape for k, v in edge_index_dict.items() }

{('venue', 'VP_Journal', 'paper'): torch.Size([2, 747053]),
 ('venue', 'VP_Patent', 'paper'): torch.Size([2, 176]),
 ('venue', 'VP_Conference', 'paper'): torch.Size([2, 61]),
 ('paper', 'PV_Journal', 'venue'): torch.Size([2, 747053]),
 ('paper', 'PV_Patent', 'venue'): torch.Size([2, 176]),
 ('paper', 'PV_Conference', 'venue'): torch.Size([2, 61]),
 ('paper', 'PF_L3', 'field'): torch.Size([2, 1710391]),
 ('paper', 'PF_L2', 'field'): torch.Size([2, 3108157]),
 ('paper', 'PF_L1', 'field'): torch.Size([2, 1332023]),
 ('paper', 'PF_L0', 'field'): torch.Size([2, 747417]),
 ('paper', 'PF_L4', 'field'): torch.Size([2, 398183]),
 ('paper', 'PF_L5', 'field'): torch.Size([2, 87674]),
 ('paper', 'PA_last', 'author'): torch.Size([2, 664930]),
 ('paper', 'PA_other', 'author'): torch.Size([2, 1634514]),
 ('paper', 'PA_first', 'author'): torch.Size([2, 630999]),
 ('field', 'FF_in', 'field'): torch.Size([2, 410909]),
 ('field', 'FP_L3', 'paper'): torch.Size([2, 1710391]),
 ('field', 'FP_L2', 'paper'): 

In [12]:
import dgl 

hg = dgl.heterograph({k: tuple(v) for k, v in edge_index_dict.items()}) 
num_nodes_dict = { ntype: hg.num_nodes(ntype) for ntype in hg.ntypes } 

num_nodes_dict 

{'author': 1097433,
 'field': 65290,
 'institution': 8868,
 'paper': 747290,
 'venue': 3013}

In [13]:
paper_feat_mat = torch.tensor(list(node_feature['paper']['emb']), dtype=torch.float32) 

paper_feat_mat.shape 

torch.Size([747290, 768])

In [ ]:
with open(os.path.expanduser('~/dataset/OAG/OAG-Chem/hg_full.dict.pkl'), 'wb') as fp: 
    pickle.dump(
        dict(
            edge_index_dict = edge_index_dict, 
            num_nodes_dict = num_nodes_dict, 
            paper_feat = paper_feat_mat, 
            paper_year = paper_year_vec, 
        ), 
        fp, 
    )